In [1]:
import pandas as pd
import numpy as np

In [2]:
R1 = pd.read_csv("H1WG.SCP1.H1.R1_5kbWindowsVsControl_Calculations.txt", sep='\t')
R1['FoldChange']= R1["log(enhancerSeq/control)"]
R1.dropna(0,inplace=True)
R1.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
1,chr1,5000,10000,3,0.0006,0.0056,-0.970037,-0.970037
2,chr1,10000,15000,471865,94.3730,37.3234,0.402867,0.402867
3,chr1,15000,20000,454345,90.8690,37.3428,0.386209,0.386209
4,chr1,20000,25000,307099,61.4198,27.5938,0.347497,0.347497
5,chr1,25000,30000,107159,21.4318,10.9186,0.292892,0.292892


In [3]:
R1_10th = np.percentile(R1["FoldChange"],90)
R1_top10 = R1[R1.FoldChange >= R1_10th]
R1_top10.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
17,chr1,85000,90000,89353,17.8706,4.5760,0.591653,0.591653
53,chr1,265000,270000,9393,1.8786,0.4848,0.588272,0.588272
68,chr1,340000,345000,14139,2.8278,0.7244,0.591470,0.591470
106,chr1,530000,535000,461152,92.2304,23.8594,0.587215,0.587215
193,chr1,965000,970000,410712,82.1424,20.3380,0.606259,0.606259


In [25]:
needMerging = [] #this ends up being an array of the index of values in the R1_top10 index, which is non-consecutive b/c sliced from the R1 index
mergedR1 = pd.DataFrame(columns=['chrom','start','end','cumulative_coverage','mean_coverage','control_mean_coverage','log(enhancerSeq/control)','FoldChange'])
for i in range(len(R1_top10.index)):
    if i==0:
        pass
    else:
        if int(R1_top10.index[i]) == int(R1_top10.index[i-1]+1):
            #print("Bordering Bins!\t" + str(i)) #lots of bordering bins
            needMerging.append(i)
        else:
            mergedR1 = mergedR1.append(R1_top10.loc[R1_top10.index[i]])
           
print(str(len(needMerging)) + " bins need to be merged.")

7466 bins need to be merged.


In [36]:
mergedR1.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
53,chr1,265000.0,270000.0,9393.0,1.8786,0.4848,0.588272,0.588272
68,chr1,340000.0,345000.0,14139.0,2.8278,0.7244,0.591470,0.591470
106,chr1,530000.0,535000.0,461152.0,92.2304,23.8594,0.587215,0.587215
193,chr1,965000.0,970000.0,410712.0,82.1424,20.3380,0.606259,0.606259
208,chr1,1040000.0,1045000.0,358521.0,71.7042,17.8808,0.603158,0.603158


In [54]:
print(R1_top10.index)
print(needMerging)


Int64Index([    17,     53,     68,    106,    193,    208,    260,    304,
               317,    334,
            ...
            618938, 618941, 618953, 618959, 618965, 619039, 619061, 619097,
            619099, 619103],
           dtype='int64', length=57126)
[27, 33, 40, 49, 79, 80, 82, 88, 95, 98, 111, 112, 120, 123, 126, 132, 144, 154, 156, 189, 212, 248, 254, 259, 275, 338, 355, 381, 383, 399, 412, 413, 415, 417, 428, 445, 478, 480, 483, 486, 487, 488, 497, 500, 505, 512, 520, 526, 527, 582, 588, 599, 601, 603, 606, 609, 611, 636, 662, 683, 691, 702, 712, 722, 724, 739, 747, 751, 759, 769, 772, 774, 782, 786, 803, 814, 816, 824, 832, 844, 847, 854, 881, 891, 894, 910, 911, 913, 927, 936, 947, 953, 957, 960, 962, 963, 986, 987, 992, 1002, 1014, 1035, 1039, 1048, 1050, 1067, 1070, 1074, 1077, 1079, 1084, 1090, 1091, 1099, 1105, 1113, 1128, 1136, 1141, 1149, 1155, 1161, 1162, 1166, 1168, 1169, 1170, 1172, 1177, 1183, 1191, 1195, 1212, 1219, 1224, 1225, 1226, 1231, 1250, 1260, 126

In [ ]:
for i in range(len(needMerging)):
    if i == 0:
        pass
    else:
        secondRow = R1_top10.loc[R1_top10.index[needMerging[i]]]
        firstRow = R1_top10.loc[R1_top10.index[needMerging[i]-1]]
        
        #get values for the new row
        mergedChrom = secondRow.chrom
        mergedStart = firstRow.start
        mergedEnd = secondRow.end
        mergedCumCoverage = (secondRow.cumulative_coverage + firstRow.cumulative_coverage)/float(2)
        mergedMeanCoverage = (secondRow.mean_coverage + firstRow.mean_coverage)/float(2)
        mergedCtrlMeanCoverage = (secondRow.control_mean_coverage + firstRow.control_mean_coverage)/float(2)
        mergedLogEnhancer = np.log10(mergedMeanCoverage/mergedCtrlMeanCoverage) #could drop this row
        mergedFoldChange = mergedLogEnhancer
        
        #add to the dataframe
        mergedR1 = mergedR1.append({'chrom':mergedChrom,'start':mergedStart,'end':mergedEnd,'cumulative_coverage':mergedCumCoverage, 'mean_coverage':mergedMeanCoverage,'control_mean_coverage':mergedCtrlMeanCoverage,'log(enhancerSeq/control)':mergedLogEnhancer,'FoldChange':mergedFoldChange}, ignore_index=True)
        
    

In [ ]:
mer